# install and import


In [1]:
!pip install -q yt-dlp soundfile spleeter moviepy ffmpeg setuptools-rust whisper librosa imagehash
# !pip install -q openai==1.36.0
# !pip install -q -U openai-whisper
# !pip install -q opencv-python
# !pip install -q nltk
# !pip install -q spacy
# !python -m spacy download en_core_web_sm
!pip install xformers
# !conda install ffmpeg -y
# # !pip install flash_attn einops

In [2]:
# ! pip install transformers
# ! pip install moviepy

In [3]:
# ! pip install 'accelerate>=0.26.0'

In [22]:
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoTokenizer, AutoModel
from transformers import CLIPProcessor, CLIPModel
from concurrent.futures import ThreadPoolExecutor
from PIL import Image, UnidentifiedImageError
from spleeter.separator import Separator
from tqdm.asyncio import tqdm_asyncio
from nltk.corpus import stopwords
from string import punctuation
from openai import AsyncOpenAI, OpenAI
import soundfile as sf
from tqdm import tqdm
import pandas as pd
import numpy as np
import yt_dlp
import whisper
import os
import csv
import librosa
import asyncio
import time
import cv2
import imagehash
import spacy
import nltk
import torch
import json
import openai
import time

import os
import torch
import csv
from tqdm import tqdm
from moviepy.editor import VideoFileClip
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

# Config & Utils


In [23]:
class Config:
    whisper_size = 'tiny'
    use_noise_reduction = False
    use_vocal_separation = False
    embed_batch_size = 16
    chat_model = 'llama3.1-8b'
    clip_model = "patrickjohncyh/fashion-clip"
    video_id = str(int(time.time()))
    output_dir = "videos_path/"
    transcripts_seconds_for_LLM = 60
    top_k = 10  # The top picked samples from the reranker
    NUM_CANDIDATES = 30  # Number of candidates to select in the first stage

    # Inputs:
    video_url = "https://www.youtube.com/watch?v=kcp2NvFH0c0"
    user_query = ""

## Load the Models


In [3]:
def load_stella_model(model_name="dunzhang/stella_en_400M_v5"):
    """
    Loads the Stella model and its corresponding tokenizer.

    Parameters:
    - model_name: The name of the model to load from Hugging Face.

    Returns:
    - embed_model: The Stella model.
    - embed_tokenizer: The tokenizer for the Stella model.
    """
    device = "cuda" if torch.cuda.is_available() else "cpu"
    embed_tokenizer = AutoTokenizer.from_pretrained(model_name)
    embed_model = AutoModel.from_pretrained(model_name,
                                            trust_remote_code=True).to(device)
    return embed_model, embed_tokenizer

In [4]:
def load_clip_model(model_name="patrickjohncyh/fashion-clip"):
    """
    Loads the CLIP model and its corresponding processor.

    Parameters:
    - model_name: The name of the model to load from Hugging Face.

    Returns:
    - clip_model: The CLIP model.
    - clip_processor: The processor for the CLIP model.
    """
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = CLIPModel.from_pretrained(model_name).to(device)
    processor = CLIPProcessor.from_pretrained(model_name)
    return model, processor

In [5]:
def load_whisper_model(model_name="openai/whisper-large-v3-turbo"):
    """
    Loads the Whisper model and its corresponding processor.

    Parameters:
    - model_name: The name of the Whisper model to load from Hugging Face.

    Returns:
    - whisper_model: The Whisper model.
    - whisper_processor: The processor for the Whisper model.
    """
    device = "cuda" if torch.cuda.is_available() else "cpu"
    torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

    # Load the Whisper model
    whisper_model = AutoModelForSpeechSeq2Seq.from_pretrained(
        model_name,
        torch_dtype=torch_dtype,
        low_cpu_mem_usage=True,
        use_safetensors=True).to(device)

    # Load the corresponding processor
    whisper_processor = AutoProcessor.from_pretrained(model_name)

    return whisper_model, whisper_processor

In [6]:
# Load the English NLP model
nltk.download('stopwords')
nlp = spacy.load("en_core_web_sm")
# Define the stopwords set
stop_words = set(stopwords.words('english'))

# Load the Stella model and tokenizer
embed_model, embed_tokenizer = load_stella_model()

# Load the CLIP model and processor
clip_model, clip_processor = load_clip_model(model_name=Config.clip_model)

# Load the Whisper model and processor
whisper_model, whisper_processor = load_whisper_model(
    model_name="openai/whisper-large-v3-turbo")

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/hamidme/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Some weights of the model checkpoint at dunzhang/stella_en_400M_v5 were not used when initializing NewModel: ['new.pooler.dense.bias', 'new.pooler.dense.weight']
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NewModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# Text-Text Matching Functions


In [7]:
def extract_keywords(text):

    # Use spaCy to parse the text
    doc = nlp(text.lower())

    # Extract keywords by filtering out stop words, punctuation, but keep numbers
    keywords = [
        token.text for token in doc
        if token.text not in stop_words and token.text not in punctuation and (
            token.is_alpha or token.like_num)
    ]

    return keywords

In [8]:
def download_youtube_video(url, output_dir):
    """
    Downloads a YouTube video in the best format and converts it to MKV format.

    Parameters:
    - url: URL of the YouTube video.
    - output_dir: Directory to save the video.

    Returns:
    - The path to the downloaded video in MKV format.
    """
    # Define the video output filename
    video_filename = f"{Config.video_id}.mkv"
    video_path = os.path.join(output_dir, video_filename)

    # yt-dlp options to download the video and convert it to MKV format
    ydl_opts = {
        'format': 'bestvideo[height<=720]+bestaudio/best[height<=720]',
        'outtmpl': output_dir,
        'merge_output_format': 'mkv',  # Ensure the output is MKV
        'noplaylist': True,
        'quiet': False,
    }

    print("Downloading YouTube video...")
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])
        print(f"Downloaded video to {video_path}")
    except Exception as e:
        print(f"An error occurred while downloading the video: {e}")
        return None

    return video_path

In [9]:
def process_youtube_video(url,
                          output_dir,
                          model_size=Config.whisper_size,
                          use_vocal_separation=False,
                          use_noise_reduction=False):
    """
    Downloads a YouTube video, extracts the audio, processes the audio, and returns the transcripts.

    Parameters:
    - url: URL of the YouTube video.
    - output_dir: Directory to save the video, audio, and transcripts.
    - model_size: Size of the Whisper model to use ("tiny", "base", "small", "medium", "large").
    - use_vocal_separation: Boolean, whether to separate music and vocals.
    - use_noise_reduction: Boolean, whether to apply noise reduction.

    Returns:
    - Path to the CSV file containing the timestamps and transcripts.
    """

    # Construct the video path using the correct extension
    video_filename = f"{Config.video_id}.mkv"
    video_path = os.path.join(output_dir, video_filename)

    # Step 2: Extract the audio from the video
    audio_filename = f"extracted_audio_{Config.video_id}.wav"
    audio_path = os.path.join(output_dir, audio_filename)

    print("Extracting audio from the video...")
    try:
        video = VideoFileClip(video_path)
        audio = video.audio
        audio.write_audiofile(audio_path)
        audio.close()
        video.close()
        print(f"Extracted audio to {audio_path}")
    except Exception as e:
        print(f"An error occurred while extracting audio: {e}")
        return None

    # Step 3: Extract transcripts using Whisper
    print("Extracting transcripts using Whisper...")

    device = "cuda" if torch.cuda.is_available() else "cpu"

    # Create the pipeline
    pipe = pipeline("automatic-speech-recognition",
                    model=whisper_model,
                    tokenizer=whisper_processor.tokenizer,
                    feature_extractor=whisper_processor.feature_extractor,
                    torch_dtype=torch.float16,
                    device=device,
                    batch_size=128,
                    chunk_length_s=30)

    # Transcribe the audio with timestamps
    result = pipe(audio_path, return_timestamps=True)

    # Process each segment's text to extract keywords and drop empty segments
    filtered_segments = []
    for segment in result['chunks']:
        processed_text = " ".join(extract_keywords(segment['text']))
        if processed_text:
            segment['text'] = processed_text
            filtered_segments.append(segment)

    # Save the filtered transcripts with timestamps to a CSV file
    transcripts_csv = os.path.join(output_dir,
                                   f"transcripts_{Config.video_id}.csv")
    with open(transcripts_csv, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Start", "End", "Text"])
        for segment in tqdm(filtered_segments, desc="Saving transcripts"):
            writer.writerow([
                segment['timestamp'][0], segment['timestamp'][1],
                segment['text']
            ])

    print(f"Transcripts saved to {transcripts_csv}")
    return transcripts_csv


In [10]:
def generate_batch_embeddings(texts):
    """
    Asynchronously generates embeddings for a batch of texts using the Stella model.

    Parameters:
    - texts: A list of strings to be embedded.

    Returns:
    - A list of embeddings for the batch.
    """
    embeddings = []

    device = "cuda" if torch.cuda.is_available() else "cpu"
    # Process embeddings in batches
    batch_size = 32  # Adjust this based on your GPU memory
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i + batch_size]
        batch_inputs = embed_tokenizer(batch,
                                       return_tensors="pt",
                                       padding=True,
                                       truncation=True,
                                       max_length=512)


        with torch.no_grad():
            batch_outputs = embed_model(**batch_inputs.to(device))

        batch_embeddings = batch_outputs.last_hidden_state[:,
                                                           0, :].cpu().numpy()
        embeddings.extend(batch_embeddings)

    return embeddings

In [11]:
def generate_and_save_transcript_embeddings(csv_file,
                                            output_file,
                                            batch_size=Config.embed_batch_size
                                            ):
    """
    Asynchronously generates embeddings for transcripts in batches and saves them to a file,
    if the embeddings file doesn't already exist. If it exists, load the embeddings from the file.

    Parameters:
    - csv_file: Path to the CSV file containing transcripts.
    - output_file: Path to save the generated embeddings.
    - batch_size: Number of transcripts to process in each batch.
    """
    # Check if the output file already exists
    print(output_file)
    print(csv_file)
    if os.path.exists(output_file):
        print(f"Embeddings file found. Loading from {output_file}.")
        df = pd.read_json(output_file, lines=True)
    else:
        print("Embeddings file not found. Generating embeddings...")
        df = pd.read_csv(csv_file)
        texts = df['Text'].tolist()
        embeddings = []

        tasks = []
        print()
        for i in range(0, len(texts), batch_size):
            batch_texts = texts[i:i + batch_size]
            tasks.append(generate_batch_embeddings(batch_texts))

        # Gather results with tqdm for progress tracking
        results = []
        for batch_embeddings in tasks:
            results.extend(batch_embeddings)

        df['Embeddings'] = results
        df.to_json(output_file, orient='records', lines=True)
        print(f"Embeddings saved to {output_file}")

    return df


def generate_candidates(user_embedding,
                        transcript_df,
                        user_query,
                        num_candidates=Config.NUM_CANDIDATES):
    """
    Generate top-n candidates based on cosine similarity and include all samples
    that contain at least one word from the user query.

    Parameters:
    - user_embedding: Embedding of the user query.
    - transcript_df: DataFrame containing the transcripts and their embeddings.
    - user_query: The original user query as a string.
    - num_candidates: Number of top candidates to select based on similarity.

    Returns:
    - A DataFrame containing the top-n candidates and those containing query words.
    """
    # Calculate similarities
    similarities = []
    for embedding in transcript_df['Embeddings']:
        similarity = cosine_similarity([user_embedding], [embedding])
        similarities.append(similarity.item())

    # Add similarities to DataFrame
    transcript_df['Similarity'] = similarities

    # Select top candidates based on similarity
    top_candidates_df = transcript_df.nlargest(num_candidates, 'Similarity')

    # Extract query words and find all samples that contain at least one query word
    query_words = set(user_query.lower().split())
    contains_query_word = transcript_df['Text'].apply(
        lambda text: any(word in query_words for word in text.lower().split()))
    word_match_candidates_df = transcript_df[contains_query_word]

    # Combine the top candidates and the word match candidates, dropping duplicates
    combined_candidates_df = pd.concat([
        top_candidates_df, word_match_candidates_df
    ]).drop_duplicates(subset=["Start", "End", "Text"]).reset_index(drop=True)

    return combined_candidates_df


In [12]:
def generate_text_embeddings(text):
    """
     generates embeddings for a given text using the Stella model.

    Parameters:
    - text: A string containing the text to be embedded.

    Returns:
    - A numpy array containing the embeddings.
    """
    # Tokenize the input text
    inputs = embed_tokenizer(text,
                             return_tensors="pt",
                             padding=True,
                             truncation=True,
                             max_length=512)

    # Generate embeddings

    device = "cuda" if torch.cuda.is_available() else "cpu"
    with torch.no_grad():
        outputs = embed_model(**inputs.to(device))

    # Use the [CLS] token embedding as the text embedding
    embedding = outputs.last_hidden_state[:, 0, :].cpu().squeeze().numpy()

    return embedding.tolist()


def calculate_cosine_similarity(embedding, candidate_embeddings):
    return cosine_similarity([embedding], candidate_embeddings).flatten()


def load_embeddings(file_path):
    """
    Load the transcript embeddings from a file.

    Parameters:
    - file_path: Path to the file containing the embeddings.

    Returns:
    - A DataFrame containing the transcripts and their embeddings.
    """
    df = pd.read_json(file_path, lines=True)
    return df


In [13]:
def call_api(api_keys, models, base_url, prompt):
    while True:
        for api_key in api_keys:
            for model in models:
                try:
                    client = OpenAI(api_key=api_key, base_url=base_url)

                    # Call OpenAI API for reranking
                    response = client.chat.completions.create(
                        model=model,
                        messages=[
                            {"role": "system", "content": "You are a helpful assistant."},
                            {"role": "user", "content": prompt}
                        ]
                    )

                    return response
                    

                except Exception as e:
                    print(f"Failed with API Key: {api_key} and Model: {model} - Error: {e}")

        # Optional delay to avoid rapid retries
        time.sleep(2)




# from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM
# import torch
# # Model name
# model_name = "microsoft/phi-4"

# # Load tokenizer
# llm_tokenizer = AutoTokenizer.from_pretrained(
#     model_name,
#     trust_remote_code=True,
#     cache_dir="/ibex/user/hamidme"
# )

# # Load model
# llm_model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     torch_dtype=torch.bfloat16,
#     low_cpu_mem_usage=True,
#     trust_remote_code=True,
#     attn_implementation="flash_attention_2",
#     device_map="cuda",
#     cache_dir="/ibex/user/hamidme"
# ).eval()




# def call_api(prompt):
#     """
#     Replaces the call to an external API with a local model inference using
#     OpenGVLab/InternVL2_5-38B-MPO. Keeps the same structure of receiving a prompt
#     and returning a 'response text'.
#     """


#     # You can prepend system instructions, etc., if needed
#     full_prompt = "You are a helpful assistant.\n" + prompt

#     # Tokenize
#     inputs = llm_tokenizer(
#         full_prompt,
#         return_tensors="pt"
#     ).to("cuda")

#     # Generate output
#     # Adjust max_new_tokens, temperature, etc. as needed
#     output_ids = llm_model.generate(
#         **inputs,
#         max_new_tokens=1024,
#         do_sample=True,
#         top_p=0.9,
#         temperature=0.7
#     )

#     # Decode the output
#     response_text = llm_tokenizer.decode(
#         output_ids[0],
#         skip_special_tokens=True
#     )

#     return response_text

In [14]:
def rerank_candidates(candidates_df, user_query, top_k=Config.top_k):
    """
    Rerank the top-n candidates using OpenAI's GPT API.

    Parameters:
    - candidates_df: DataFrame containing the top candidates.
    - user_query: The user's search query.
    - top_k: Number of top results to return after reranking.

    Returns:
    - A DataFrame containing the top-k reranked candidates.
    """
    # Prepare the prompt for OpenAI API with few-shot examples
    prompt = (
        f"The following are transcript segments from a video. The user entered the query: '{user_query}'. "
        "Please rerank these segments based on their words existence in the query. "
        "Return the indices of the segments in descending order of relevance in a list format like ['5','12','2','7','3',...].\n\n"
    )

    # Add examples
    prompt += (
        "Example 1:\n"
        "Query: 'drug discovery'\n"
        "Segments: ['This research discusses the importance of drug discovery.', "
        "'This is unrelated text.', 'A discussion on molecular biology and drugs.']\n"
        "Return: ['0','2','1']\n\n"
        "Example 2:\n"
        "Query: 'climate change'\n"
        "Segments: ['The effects of climate change are far-reaching.', "
        "'Discussing climate inside the room', 'Random talk about sports.']\n"
        "Return: ['0','1','2']\n\n")

    while True:
        # Add actual candidates
        prompt += f"Query: '{user_query}'\nSegments:\n"
        for i, row in candidates_df.iterrows():
            prompt += f"'{i}: {row['Text']}'\n"

        prompt += "\nReturn the indices:"

        #  # Call the local model instead of an external API
        # response_text = call_api(prompt).strip()

        # # Attempt to parse the list of indices from the response text
        # try:
        #     start = response_text.find("[")
        #     end = response_text.find("]", start)
        #     reranked_indices = response_text[start:end + 1]

        #     reranked_indices = eval(reranked_indices)  # Should be something like ['5', '12', '2', ...]
        #     top_k_df = candidates_df.iloc[[int(index) for index in reranked_indices]]
        #     return top_k_df

        # except Exception as e:
        #     print(f"Parsing the reranked list failed. Retrying... Error: {e}")
        #     # Optional delay to avoid rapid retries
        #     time.sleep(2)
        #     continue

        api_keys = ['######', '######', '######', '######', '######', '######', '######', '######', '######', '######', '######', '######', '######', '######', '######', '######', '######', '######']
        api_keys = api_keys[2:]
        # List of models to try
        models = ["gemini-2.0-flash-exp"]

        base_url = 'https://generativelanguage.googleapis.com/v1beta/openai/'

        response = call_api(api_keys, models, base_url, prompt)

        response_text = response.choices[0].message.content.strip()

        # Extract the list of indices from the response text

        start = response_text.find("[")
        end = response_text.find("]", start)
        reranked_indices = response_text[start:end + 1]

        try:
            reranked_indices = eval(reranked_indices)
            top_k_df = candidates_df.iloc[[
                int(index) for index in reranked_indices[:top_k]
            ]]
            return top_k_df
        except Exception as e:
            continue


In [15]:
def extract_transcript_for_one_minute(
        chosen_index,
        transcript_df,
        time_column='Start',
        duration=Config.transcripts_seconds_for_LLM):
    """
    Extracts the transcript for the next 1 minute starting from the chosen index.

    Parameters:
    - chosen_index: The index of the chosen segment.
    - transcript_df: DataFrame containing the transcripts.
    - time_column: The column name that contains the timestamp.
    - duration: Duration in seconds to extract the transcript.

    Returns:
    - A string containing the concatenated transcript for the next 1 minute.
    """
    start_time = transcript_df.iloc[chosen_index][time_column]
    end_time = start_time + duration

    # Filter segments that fall within the 1-minute window
    selected_segments = transcript_df[
        (transcript_df[time_column] >= start_time)
        & (transcript_df[time_column] < end_time)]

    # Concatenate the text from the selected segments
    combined_text = " ".join(selected_segments['Text'].tolist())

    return combined_text

In [16]:
async def generate_llm_answer(top_k_file, csv_file, user_question):
    """
    Generates a brief answer using GPT-4 based on the user's question and the top similar transcript segment.

    Parameters:
    - top_k_file: Path to the CSV file containing the top-k similar segments.
    - csv_file: Path to the CSV file containing transcripts.
    - user_question: The user's question.

    Returns:
    - A string containing the LLM-generated answer.
    """
    # Set your OpenAI API key
    api_key = Config.together_api_key
    client = AsyncOpenAI(api_key=api_key,
                         base_url='https://api.together.xyz/v1')

    # Load the top-k matches
    top_k_df = pd.read_csv(top_k_file)

    # Select the top index
    chosen_index = top_k_df.index[
        0]  # Assuming 'index' is the column where the original indices are saved

    # Load the transcripts
    transcript_df = pd.read_csv(csv_file)

    # Extract the transcript for the next 1 minute starting from the chosen index
    context_text = extract_transcript_for_one_minute(chosen_index,
                                                     transcript_df)

    # Create the prompt for LLAMA 3.1
    prompt = f"Based on the following context: {context_text}\n\nAnswer the following question briefly in less than 5 phrases (if there is no enough context, you can explain it from your knowledge without mentioning that): {user_question}"

    # Generate the answer using LLAMA 3.1
    response = await client.chat.completions.create(
        model=Config.chat_model,
        messages=[{
            "role": "system",
            "content": "You are a helpful assistant."
        }, {
            "role": "user",
            "content": prompt
        }])

    answer = response.choices[0].message.content
    return answer

# Text-Image matching


In [17]:
import cv2
import numpy as np
from PIL import Image
import imagehash


def extract_frames(video_path, sampling_frequency, similarity_threshold=0.95):
    print("Starting frame extraction...")
    frames_with_timestamps = []

    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)

    if fps == 0:
        raise ValueError("Could not retrieve FPS from the video.")

    frame_interval = int(fps * sampling_frequency)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    count = 0
    previous_hash = None

    while count < total_frames:
        # Skip frames efficiently using cap.grab()
        for _ in range(frame_interval - 1):
            cap.grab()
            count += 1
            if count >= total_frames:
                break

        # Read the frame you need
        success, frame = cap.read()
        count += 1
        if not success:
            break

        timestamp = count / fps

        # Resize and process the frame for perceptual hashing
        small_frame = cv2.resize(
            frame, (64, 64))  # Resize to reduce computational cost
        pil_image = Image.fromarray(
            cv2.cvtColor(small_frame,
                         cv2.COLOR_BGR2RGB))  # Convert to PIL format
        current_hash = imagehash.phash(pil_image)  # Compute perceptual hash

        # Compare with the previous hash
        if previous_hash is None or (1 - (current_hash - previous_hash) / len(
                current_hash.hash)**2) < similarity_threshold:
            frames_with_timestamps.append((timestamp, frame))
            previous_hash = current_hash

    cap.release()
    print("Frame extraction completed.")
    return frames_with_timestamps

In [18]:
def precompute_frame_embeddings(
        frames,
        model,
        processor,
        batch_size=16,
        output_file=f"frame_embeddings_{Config.video_id}.json"):
    frame_embeddings = []

    print("Generating embeddings for frames in batches...")

    # Split frames and timestamps into batches
    batches = [
        frames[i:i + batch_size] for i in range(0, len(frames), batch_size)
    ]

    for batch in tqdm(batches, desc="Processing frames"):
        try:
            images = [
                Image.fromarray(cv2.cvtColor(frame,
                                             cv2.COLOR_BGR2RGB)).convert('RGB')
                for timestamp, frame in batch
            ]
            image_inputs = processor(images=images,
                                     return_tensors="pt",
                                     padding=True).to(model.device)

            with torch.no_grad():
                image_features = model.get_image_features(**image_inputs)
                image_features = image_features / image_features.norm(
                    p=2, dim=-1, keepdim=True)

            for (timestamp, _), features in zip(batch, image_features):
                frame_embeddings.append(
                    (timestamp, features.cpu().numpy().tolist()))
        except Exception as e:
            print(f"Error processing batch: {e}")
            continue

    with open(output_file, "w") as f:
        json.dump(frame_embeddings, f)

    print(f"Frame embeddings saved to {output_file}.")

In [19]:
def find_similar_frames(
        user_prompt,
        model,
        processor,
        embeddings_file=f"frame_embeddings_{Config.video_id}.json",
        top_k=5,
        relevance_threshold=0.1):
    print("Generating text embeddings...")
    text_inputs = processor(text=user_prompt,
                            return_tensors="pt",
                            padding=True).to(model.device)
    with torch.no_grad():
        text_features = model.get_text_features(**text_inputs)
        text_features = text_features / text_features.norm(
            p=2, dim=-1, keepdim=True)

    with open(embeddings_file, "r") as f:
        frame_embeddings = json.load(f)

    results = []
    for timestamp, image_features in frame_embeddings:
        image_features = torch.tensor(image_features).to(model.device)
        similarity = torch.matmul(text_features, image_features.T)
        results.append((timestamp, similarity.item()))

    results.sort(key=lambda x: x[1], reverse=True)
    top_k_frames = results[:top_k]

    if top_k_frames and top_k_frames[0][1] < relevance_threshold:
        print("No relevant images found for the given query.")
        return []

    print("Top frames selected.")
    return top_k_frames

In [20]:
def save_top_frames_to_csv(top_frames,
                           output_file=f"top_frames_{Config.video_id}.csv"):
    """
    Saves the top frames to a CSV file.

    Parameters:
    - top_frames: A list of tuples (timestamp, frame_path, similarity).
    - output_file: The output CSV file path.
    """
    with open(output_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Timestamp", "Similarity"])
        for frame in top_frames:
            writer.writerow(frame)

    print(f"Top frames saved to {output_file}.")


import pandas as pd


def find_intersections_and_return_timestamp(images_matches_csv,
                                            text_matches_csv,
                                            text_similarity_threshold=0.8,
                                            image_similarity_threshold=0.3):
    """
    Finds the most appropriate timestamp by merging the top-k text and image matches, considering similarity thresholds.

    Parameters:
    - images_matches_csv: Path to the CSV file containing the image matches.
    - text_matches_csv: Path to the CSV file containing the text matches.
    - text_similarity_threshold: Minimum similarity score to consider the text match relevant.
    - image_similarity_threshold: Minimum similarity score to consider the image match relevant.

    Returns:
    - The appropriate timestamp or "Irrelevant question" if the question is deemed irrelevant.
    """
    top_frames = pd.read_csv(images_matches_csv).values.tolist()
    text_matches_df = pd.read_csv(text_matches_csv)
    text_matches = list(text_matches_df[['Start', 'End', 'Text',
                                         'Similarity']].itertuples(index=False,
                                                                   name=None))

    top_image_timestamp = top_frames[0][0]
    top_text_timestamp = text_matches[0][0]

    # Calculate the time difference between the top text and image matches
    time_difference = abs(top_image_timestamp - top_text_timestamp)

    # If the time difference is less than 60 seconds, return the text timestamp
    if time_difference <= 60:
        return top_text_timestamp

    # If the time difference is greater than 60 seconds
    # Case 2: If there is a valid image match, return the top image timestamp
    if top_frames[0][1] >= image_similarity_threshold:
        return top_image_timestamp

    # Case 3: If no valid image match, return the top text timestamp
    return top_text_timestamp

In [24]:
import torch
from transformers import (
    CLIPTokenizer,
    CLIPTextModelWithProjection,
    CLIPVisionModelWithProjection,
)
from torchvision.transforms import (
    Compose, Resize, CenterCrop, ToTensor,
    Normalize, InterpolationMode
)


device = "cuda" if torch.cuda.is_available() else "cpu"

# TEXT PART
text_model = CLIPTextModelWithProjection.from_pretrained("Searchium-ai/clip4clip-webvid150k").to(device).eval()
tokenizer = CLIPTokenizer.from_pretrained("Searchium-ai/clip4clip-webvid150k")

# VIDEO PART
vision_model = CLIPVisionModelWithProjection.from_pretrained("Searchium-ai/clip4clip-webvid150k").to(device).eval()

frame_size = 224  # or whatever you want
preprocess_transform = Compose([
    Resize(frame_size, interpolation=InterpolationMode.BICUBIC),
    CenterCrop(frame_size),
    lambda image: image.convert("RGB"),  # ensure RGB
    ToTensor(),
    Normalize(
        mean=(0.48145466, 0.4578275, 0.40821073),
        std=(0.26862954, 0.26130258, 0.27577711)
    ),
])

def get_text_embedding_for_video_query(user_query: str):
    """
    Encodes a single text string using CLIPTextModelWithProjection from 
    'Searchium-ai/clip4clip-webvid150k', returning a normalized text embedding (1, hidden_dim).
    """
    inputs = tokenizer(user_query, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        text_out = text_model(**inputs)  # returns dict with 'text_embeds'
        text_embeds = text_out["text_embeds"]  # shape: (batch=1, hidden_dim)
        # Normalize
        text_embeds = text_embeds / text_embeds.norm(dim=-1, keepdim=True)
    return text_embeds  # shape: (1, hidden_dim)

import cv2
import torch
import numpy as np
from PIL import Image
from torchvision.transforms import (
    Compose,
    Resize,
    CenterCrop,
    ToTensor,
    Normalize,
    InterpolationMode,
)

def video2image(video_path, frame_rate=1.0, size=224):
    """
    Extract frames from 'video_path' at 'frame_rate' frames/sec, 
    transform each to (3, size, size), and stack them into a single tensor: (num_frames, 3, size, size).
    This version avoids cap.set(...) for every frame, making decoding much faster.
    """

    # Pre-build the transform pipeline (so it's not rebuilt for every frame)
    transform = Compose([
        Resize(size, interpolation=InterpolationMode.BICUBIC),
        CenterCrop(size),
        lambda image: image.convert("RGB"),
        ToTensor(),
        Normalize(
            mean=(0.48145466, 0.4578275, 0.40821073),
            std=(0.26862954, 0.26130258, 0.27577711),
        ),
    ])

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError(f"Cannot open {video_path}")

    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    if fps < 1:
        cap.release()
        raise ValueError(f"Invalid FPS in video: {video_path}")

    # Duration in seconds
    total_duration = frame_count / fps  # float
    # We'll sample frames at 'frame_rate' frames/sec
    # so we want 1 frame every (fps / frame_rate) original frames
    interval = fps / frame_rate

    # Calculate all the frame indices we actually want
    # e.g. if frame_rate=1.0, we take 1 frame per second => indices [0, fps, 2*fps, ...].
    frames_idx = np.arange(0, frame_count, interval)
    frames_idx = np.floor(frames_idx).astype(int)  # ensure integer frame indices

    # Convert indices to a set for O(1) membership checks
    keep_idx = set(frames_idx.tolist())

    images = []
    current_idx = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            # no more frames or error reading
            break

        # If this frame index is in keep_idx, we process it
        if current_idx in keep_idx:
            # Convert BGR to RGB
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            # To PIL
            pil_frame = Image.fromarray(frame_rgb)
            # Apply transforms
            processed = transform(pil_frame)
            images.append(processed)

        current_idx += 1
        if current_idx >= frame_count:
            break

    cap.release()

    # If no frames were captured (e.g., frame_rate was very high or video is too short),
    # return a dummy tensor of shape (1, 3, size, size).
    if len(images) == 0:
        return torch.zeros((1, 3, size, size), dtype=torch.float32)

    # Stack into (num_frames, 3, size, size)
    return torch.stack(images, dim=0)



def encode_video_clip(clip_path: str):
    """
    1. Extract frames from the clip using video2image(...).
    2. Pass them through CLIPVisionModelWithProjection (Searchium-ai/clip4clip-webvid150k).
    3. Normalize each frame embed, average them, normalize again => final (1, hidden_dim).
    """
    # 1) Frames
    frames_tensor = video2image(clip_path, frame_rate=1.0, size=224).to(device)
    # shape: (num_frames, 3, 224, 224)
    
    # 2) Forward pass
    # The model expects a batch shape => (batch, channels, height, width).
    # So let's add a batch dimension for frames => (num_frames, 3, 224,224)
    with torch.no_grad():
        outputs = vision_model(frames_tensor)  
        # outputs["image_embeds"] => shape: (num_frames, hidden_dim)
    
    image_embeds = outputs["image_embeds"]
    # Normalize each frame’s embed
    image_embeds = image_embeds / image_embeds.norm(dim=-1, keepdim=True)
    
    # 3) Average across frames => (hidden_dim,)
    mean_embed = image_embeds.mean(dim=0, keepdim=True)  # shape: (1, hidden_dim)
    # Final normalize
    mean_embed = mean_embed / mean_embed.norm(dim=-1, keepdim=True)
    
    return mean_embed  # shape: (1, hidden_dim)

import os
import ffmpeg
import torch
import pandas as pd


def image_pipeline(input_video_path, segments, user_query, top_k=10):
    """
    1. user_query -> text embed
    2. For each segment: cut => encode => dot product => store similarity
    3. Sort & return top_k

    Now with caching of (start_s, end_s) -> embedding
    """
    # 1) text embed
    text_emb = get_text_embedding_for_video_query(user_query)  # shape: (1, hidden_dim)

    # 2) Load segments
    segments = pd.read_csv(segments)
    
    results = []
    for i in range(len(segments)):
        start_s = segments.loc[i,'Start']
        end_s   = segments.loc[i,'End']

        # Check if embedding is in cache
        if (start_s, end_s) in video_embedding_cache:
            # Reuse the cached embedding
            video_emb = video_embedding_cache[(start_s, end_s)]
        else:
            # Not cached -> cut and encode
            clip_path = f"videos_path/temp_clip_{i}.mp4"

            # Cut the segment (copy the exact frames from source)
            (
                ffmpeg
                .input(input_video_path, ss=start_s, to=end_s)
                .output(clip_path, codec='copy', shortest=None)
                .overwrite_output()
                .run(quiet=True)
            )

            # Encode => get video embedding
            video_emb = encode_video_clip(clip_path)  # shape: (1, hidden_dim)

            # Save to cache
            video_embedding_cache[(start_s, end_s)] = video_emb

            # Optional cleanup
            if os.path.exists(clip_path):
                os.remove(clip_path)

        # Compute similarity (dot product)
        sim_score = torch.matmul(text_emb, video_emb.T).item()

        results.append({
            'Start': start_s,
            'End':   end_s,
            'similarity_image': sim_score
        })

    # 3) Sort results
    df = pd.DataFrame(results)
    df = df.sort_values('similarity_image', ascending=False)
    df = df.head(top_k).reset_index(drop=True)

    return df


# def image_pipeline(input_video_path, segments, user_query, top_k=10):
#     """
#     1. user_query -> text embed
#     2. For each segment: cut => encode => dot product => store similarity
#     3. Sort & return top_k
#     """
#     # 1) text embed
#     text_emb = get_text_embedding_for_video_query(user_query)  # shape: (1,hidden_dim)

#     segments = pd.read_csv(segments)
    
#     results = []
#     for i in range(len(segments)):
        
#         start_s = segments.loc[i,'Start']
#         end_s   = segments.loc[i,'End']
#         clip_path = f"videos_path/temp_clip_{i}.mp4"

#         # Cut the segment
#         (
#             ffmpeg
#             .input(input_video_path, ss=start_s, to=end_s)
#             .output(clip_path, codec='copy', shortest=None)
#             .overwrite_output()
#             .run(quiet=True)
#         )

#         # 2) encode
#         video_emb = encode_video_clip(clip_path)  # shape (1,hidden_dim)

#         # dot product => similarity
#         sim = torch.matmul(text_emb, video_emb.T)  # shape: (1,1)
#         sim_score = sim.item()

#         results.append({
#             'Start': start_s,
#             'End':   end_s,
#             'similarity_image': sim_score
#         })

#         # optional cleanup
#         if os.path.exists(clip_path):
#             os.remove(clip_path)

#     # 3) sort, keep top_k
#     df = pd.DataFrame(results)
#     df = df.sort_values('similarity_image', ascending=False)
#     # if len(df) > top_k:
#     #     df = df.head(top_k)
#     return df.reset_index(drop=True)

def merge_by_intersect(text_df, image_df, top_k=10):
    text_top = pd.read_csv(text_df).head(top_k).copy()
    image_top = pd.read_csv(image_df).head(top_k).copy()
    merged = pd.merge(text_top, image_top, on=["Start"], how="inner")
    merged["avg_similarity"] = (merged["Similarity"] + merged["similarity_image"])/2
    merged = merged.sort_values("avg_similarity", ascending=False).reset_index(drop=True)
    return merged


# Main program


In [25]:
# import time
# from tqdm import tqdm
# import csv
# import json
# import os
# import pandas as pd
# import shutil

# final_answer = {'timestamps': [], 'LLM_answer': ''}


# def find_intersections_and_return_timestamp_iteratively(
#         images_matches_csv,
#         text_matches_csv,
#         text_similarity_threshold=0.8,
#         image_similarity_threshold=0.3):
#     """
#     Iteratively finds the most appropriate timestamps by merging top-k text and image matches, considering similarity thresholds.
#     The function will continue until one of the text matches or images matches gets zeroed.
#     The remaining samples will be concatenated.

#     Parameters:
#     - images_matches_csv: Path to the CSV file containing the image matches.
#     - text_matches_csv: Path to the CSV file containing the text matches.
#     - text_similarity_threshold: Minimum similarity score to consider the text match relevant.
#     - image_similarity_threshold: Minimum similarity score to consider the image match relevant.

#     Returns:
#     - A dictionary containing the final selected timestamps.
#     """

#     # Load the matches from CSV files
#     top_frames = pd.read_csv(images_matches_csv).values.tolist()
#     text_matches_df = pd.read_csv(text_matches_csv)
#     text_matches = list(text_matches_df[['Start', 'End', 'Text',
#                                          'Similarity']].itertuples(index=False,
#                                                                    name=None))

#     selected_timestamps = []

#     while text_matches and top_frames:
#         top_image_timestamp = top_frames[0][0]
#         top_text_timestamp = text_matches[0][0]

#         # Calculate the time difference between the top text and image matches
#         time_difference = abs(top_image_timestamp - top_text_timestamp)

#         # If the time difference is less than or equal to 60 seconds, select the text timestamp
#         if time_difference <= 60:
#             selected_timestamps.append(top_text_timestamp)
#             # Remove the matched top items and continue
#             text_matches.pop(0)
#             top_frames.pop(0)
#         else:
#             # If the time difference is greater than 60 seconds, check similarity thresholds
#             if top_frames[0][1] >= image_similarity_threshold:
#                 selected_timestamps.append(top_image_timestamp)
#                 top_frames.pop(0)
#             elif text_matches[0][3] >= text_similarity_threshold:
#                 selected_timestamps.append(top_text_timestamp)
#                 text_matches.pop(0)
#             else:
#                 break  # Break the loop if no valid match is found

#     # If either list is exhausted, concatenate the remaining timestamps
#     remaining_timestamps = [item[0] for item in text_matches[:5]
#                             ] + [item[0] for item in top_frames[:5]]
#     selected_timestamps.extend(remaining_timestamps)

#     return {"final_selected_timestamps": selected_timestamps[:10]}


In [26]:
# Main processing function
def main(url, user_query):
    final_answer = {'timestamps': [], 'LLM_answer': ''}
    start_time = time.time()  # Start timing the entire process

    # Define paths for all the necessary files
    video_path = Config.output_dir + Config.video_id + '.mkv'
    transcripts_csv = os.path.join(Config.output_dir,
                                   f"transcripts_{Config.video_id}.csv")
    transcript_embeddings_file = os.path.join(
        Config.output_dir, f"transcript_embeddings_{Config.video_id}.jsonl")
    embeddings_file = os.path.join(Config.output_dir,
                                   f"frame_embeddings_{Config.video_id}.json")
    text_matches_csv = os.path.join(Config.output_dir,
                                    f"top_k_matches_{Config.video_id}.csv")
    images_matches_csv = os.path.join(Config.output_dir,
                                      f"top_frames_{Config.video_id}.csv")

    # Step 1: Download the video if it doesn't exist
    # step_start = time.time()
    # if not os.path.exists(video_path):
    #     download_youtube_video(url, video_path)

    # Check if the video was successfully downloaded
    # if not os.path.exists(video_path):
    #     print(
    #         f"Video download failed or video does not exist at {video_path}. Skipping this video."
    #     )
    #     return None, []  # Skip further processing for this video

    # print(f"Step 1 completed in {time.time() - step_start:.2f} seconds")

    # Step 2: Extract the transcripts if the CSV doesn't exist
    step_start = time.time()
    # if not os.path.exists(transcripts_csv):
        # transcripts_csv = process_youtube_video(
        #     url,
        #     Config.output_dir,
        #     model_size=Config.whisper_size,
        #     use_vocal_separation=Config.use_vocal_separation,
        #     use_noise_reduction=Config.use_noise_reduction)
    print(f"Step 2 completed in {time.time() - step_start:.2f} seconds")

    # Step 3: Generate or load transcript embeddings
    step_start = time.time()
    if not os.path.exists(transcript_embeddings_file):
        generate_and_save_transcript_embeddings(transcripts_csv,
                                                transcript_embeddings_file)
    print(f"Step 3 completed in {time.time() - step_start:.2f} seconds")

    # Step 4: Candidates Generation
    step_start = time.time()
    transcript_df = load_embeddings(transcript_embeddings_file)
    user_embedding = generate_text_embeddings(user_query)
    candidates_df = generate_candidates(user_embedding,
                                        transcript_df,
                                        user_query,
                                        num_candidates=Config.NUM_CANDIDATES)
    

    print(f"Step 4 completed in {time.time() - step_start:.2f} seconds")

    # Step 5: Reranking the top candidates
    step_start = time.time()
    top_k_text = rerank_candidates(candidates_df, user_query, top_k=Config.top_k)
    print(f"Step 5 completed in {time.time() - step_start:.2f} seconds")

    # Save the reranked top-k results to a CSV file
    top_k_text.to_csv(text_matches_csv, index=False)

    # Step 6: Precompute frame embeddings if they don't exist
    step_start = time.time()
    # if not os.path.exists(embeddings_file):
    #     frames_with_timestamps = extract_frames(video_path,
    #                                             sampling_frequency=1)
    #     print(f"Step 5.5 completed in {time.time() - step_start:.2f} seconds")
    #     step_start = time.time()
    #     precompute_frame_embeddings(frames_with_timestamps,
    #                                 clip_model,
    #                                 clip_processor,
    #                                 output_file=embeddings_file)
    try:
        top_k_clips = image_pipeline(video_path, transcripts_csv, user_query)
        top_k_clips.to_csv(images_matches_csv, index=False)
    except:
        top_k_clips = pd.DataFrame()
    print(top_k_clips)
    print(f"Step 6 completed in {time.time() - step_start:.2f} seconds")

    # Step 7: Find similar frames based on the user query
    step_start = time.time()
    # top_frames = find_similar_frames(user_query,
    #                                  clip_model,
    #                                  clip_processor,
    #                                  embeddings_file=embeddings_file,
    #                                  top_k=Config.top_k)
    # save_top_frames_to_csv(top_frames, output_file=images_matches_csv)
    print(f"Step 7 completed in {time.time() - step_start:.2f} seconds")

    # Step 8: Merge the top k from text and images, get the timestamp and the LLM answer
    step_start = time.time()
    # result = find_intersections_and_return_timestamp_iteratively(
    #     images_matches_csv, text_matches_csv)
    try:
        result = merge_by_intersect(text_matches_csv, images_matches_csv, top_k=10)
        print(result)
        final_answer['timestamps'] = "[" + result["Start"].astype(str) + "," + result["End_x"].astype(str) + "]"
    except:
        final_answer['timestamps'] = pd.Series([])

    # earliest_start_timestamp = final_answer['timestamps'][0]
    # earliest_end_timestamp = final_answer['timestamps'][1]

    print(f"Step 8 completed in {time.time() - step_start:.2f} seconds")

    # Return the earliest timestamp and the top 10 timestamps
    # return earliest_start_timestamp, earliest_end_timestamp, final_answer['timestamps']
    return final_answer['timestamps']


## Benchmarking


In [27]:
# Function to check if a timestamp falls within the correct segment
import numpy as np

def is_correct_timestamp(pred_start, pred_end, correct_segment):
    """
    Checks if the predicted clip [pred_start, pred_end] overlaps at least
    50% with the ground-truth clip [gt_start, gt_end] (i.e. `correct_segment`).

    Returns:
        1 if the overlap is >= 50% of the predicted clip's duration,
        0 otherwise.
    """

    gt_start, gt_end = correct_segment

    # Calculate intersection boundaries
    intersection_start = max(pred_start, gt_start)
    intersection_end   = min(pred_end, gt_end)

    # Lengths
    pred_length = max(0, pred_end - pred_start)
    intersection_length = max(0, intersection_end - intersection_start)

    # Edge case: if predicted clip or intersection is zero/negative, return 0
    if pred_length == 0 or intersection_length <= 0:
        return 0

    # Fraction of predicted clip that overlaps the ground-truth
    overlap_fraction = intersection_length / pred_length

    # Check if at least 50% of the predicted clip overlaps
    return int(overlap_fraction >= 0.5)



# Function to calculate Recall@k
def calculate_recall_at_k(results, k):
    correct_predictions = 0

    for result in results:
        # Parse the 'segment' (the ground-truth [start, end]) 
        # and 'top_k_timestamps' (the predicted clips).
        gt_segment = eval(result['segment'])  
        # e.g. [47, 60]

        top_k_list = eval(result['top_k_timestamps']) 
        # e.g. [ [47.2, 59.9], [67,89], [91,98] ] 
        # or maybe just one predicted clip [start, end].

        # Check if ANY of the top-k predicted clips is correct
        # (meaning at least 50% intersection, or your chosen criterion).
        # If yes, we consider this entire result "correct."
        is_correct_for_this_result = any(
            is_correct_timestamp(eval(pred_clip)[0], eval(pred_clip)[1], gt_segment)
            for pred_clip in top_k_list[:k]
        )

        if is_correct_for_this_result:
            correct_predictions += 1

    return correct_predictions / len(results)



def process_video(url, user_queries, video_id):
    # Define paths for all the necessary files
    
    video_path = os.path.join(Config.output_dir, f"{Config.video_id}.mkv")
    audio_path = os.path.join(Config.output_dir,
                              f"extracted_audio_{Config.video_id}.wav")
    transcripts_csv = os.path.join(Config.output_dir,
                                   f"transcripts_{Config.video_id}.csv")
    transcript_embeddings_file = os.path.join(
        Config.output_dir, f"transcript_embeddings_{Config.video_id}.jsonl")
    embeddings_file = os.path.join(Config.output_dir,
                                   f"frame_embeddings_{Config.video_id}.json")
    text_matches_csv = os.path.join(Config.output_dir,
                                    f"top_k_matches_{Config.video_id}.csv")
    images_matches_csv = os.path.join(Config.output_dir,
                                      f"top_frames_{Config.video_id}.csv")

    # Ensure video is processed only once
    main(url, user_queries[0]
         [0])  # Process video with the first query to generate all embeddings

    for user_query, correct_segment in user_queries:
        # Replace Config.user_query with user_query from loop
        Config.user_query = user_query
        Config.video_url = url

        # Call the main pipeline with the specific user query
        # earliest_start_timestamp, earliest_end_timestamp, top_k_timestamps = main(url, user_query)
        top_k_timestamps = main(url, user_query)

        # Record the final answer
        yield {
            'video_id': video_id,
            'query': user_query,
            'segment': correct_segment,
            # 'predicted_start_timestamp': earliest_start_timestamp,
            # 'predicted_end_timestamp': earliest_end_timestamp,
            'top_k_timestamps': list(top_k_timestamps.values)  # Collect top 10 timestamps
        }

    # Clean up: delete the video and related files after processing
    video_related_files = [
        video_path, audio_path, transcripts_csv, transcript_embeddings_file,
        embeddings_file, text_matches_csv, images_matches_csv
    ]
    for file_path in video_related_files:
        if os.path.exists(file_path):
            os.remove(file_path)


In [28]:
import csv

# Load YouCookII data
with open('youcookii_annotations_trainval.json', 'r') as file:
    data = json.load(file)

# Filter for validation samples
validation_data = {
    k: v
    for k, v in data['database'].items() if v['subset'] == 'validation'
}

# Initialize CSV file for storing results
output_csv = 'results.csv'
if not os.path.exists(output_csv):
    with open(output_csv, 'w', newline='') as csvfile:
        fieldnames = [
            'video_id', 'query', 'segment',
            # 'predicted_start_timestamp','predicted_end_timestamp',
            'top_k_timestamps'
        ]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

# Load existing results to skip already processed videos
processed_videos = []
if os.path.exists(output_csv):
    existing_results = pd.read_csv(output_csv)
    processed_videos = existing_results['video_id'].unique().tolist()

# Process each video in the validation set
for video_id, video_data in tqdm(validation_data.items(),
                                 desc="Processing videos"):
    if video_id in processed_videos:
        continue  # Skip already processed videos

    video_embedding_cache = {}  # key: (start_s, end_s), value: torch.Tensor
    video_url = video_data['video_url']
    Config.video_id = str(int(time.time()))
    user_queries = [(ann['sentence'], ann['segment'])
                    for ann in video_data['annotations']]

    video_path = Config.output_dir + Config.video_id + '.mkv'
    download_youtube_video(video_url, video_path)

    transcripts_csv = process_youtube_video(
            video_url,
            Config.output_dir,
            model_size=Config.whisper_size,
            use_vocal_separation=Config.use_vocal_separation,
            use_noise_reduction=Config.use_noise_reduction)
    if not transcripts_csv:
        continue

    # user_queries = []
    # with open(transcripts_csv, newline='', encoding='utf-8') as f:
    #     reader = csv.DictReader(f)
    #     for row in reader:
    #         text = row["Text"]
    #         start = float(row["Start"])
    #         end = float(row["End"])
    #         user_queries.append((text, [start, end]))

    # Process the video and queries
    for result in process_video(video_url, user_queries, video_id):
        # Write results to CSV
        with open(output_csv, 'a', newline='') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=result.keys())
            writer.writerow(result)

# Load results and calculate Recall@k
results = pd.read_csv(output_csv)

recall_at_1 = calculate_recall_at_k(results.to_dict('records'), 1)
recall_at_5 = calculate_recall_at_k(results.to_dict('records'), 5)
recall_at_10 = calculate_recall_at_k(results.to_dict('records'), 10)

print(f"Recall@1: {recall_at_1:.2f}")
print(f"Recall@5: {recall_at_5:.2f}")
print(f"Recall@10: {recall_at_10:.2f}")

Processing videos:   0%|                                                                       | 0/457 [00:00<?, ?it/s]

[youtube] Extracting URL: https://www.youtube.com/watch?v=VH0SmCfAov4
[youtube] VH0SmCfAov4: Downloading webpage
[youtube] VH0SmCfAov4: Downloading ios player API JSON
[youtube] VH0SmCfAov4: Downloading web creator player API JSON


ERROR: [youtube] VH0SmCfAov4: Private video. Sign in if you've been granted access to this video


An error occurred while downloading the video: ERROR: [youtube] VH0SmCfAov4: Private video. Sign in if you've been granted access to this video
Extracting audio from the video...


Processing videos:   3%|██                                                            | 15/457 [00:01<00:55,  8.04it/s]

...


Saving transcripts: 100%|█████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 17848.10it/s]


Transcripts saved to videos_path/transcripts_1736923683.csv
Step 2 completed in 0.00 seconds
videos_path/transcript_embeddings_1736923683.jsonl
videos_path/transcripts_1736923683.csv
Embeddings file not found. Generating embeddings...

Embeddings saved to videos_path/transcript_embeddings_1736923683.jsonl
Step 3 completed in 0.02 seconds
Step 4 completed in 0.03 seconds
Step 5 completed in 1.43 seconds
   Start     End  similarity_image
0    0.0  121.94          0.290561
Step 6 completed in 11.30 seconds
Step 7 completed in 0.00 seconds
   Start   End_x               Text  \
0      0  121.94  music thank thank   

                                          Embeddings  Similarity   End_y  \
0  [-0.255751282, -1.1835052967, -0.5935797095, 0...    0.349649  121.94   

   similarity_image  avg_similarity  
0          0.290561        0.320105  
Step 8 completed in 0.01 seconds
Step 2 completed in 0.00 seconds
Step 3 completed in 0.00 seconds
Step 4 completed in 0.03 seconds
Step 5 completed 

Processing videos: 100%|███████████████████████████████████████████████████████████| 457/457 [3:35:09<00:00, 28.25s/it]

Step 5 completed in 1.45 seconds
   Start     End  similarity_image
0    0.0  121.94          0.279894
Step 6 completed in 0.01 seconds
Step 7 completed in 0.00 seconds
Empty DataFrame
Columns: [End_x, Text, Embeddings, Similarity, Start, End_y, similarity_image, avg_similarity]
Index: []
Step 8 completed in 0.01 seconds


Recall@1: 0.32
Recall@5: 0.47
Recall@10: 0.47


In [32]:
import pandas as pd


# Function to check if a timestamp falls within the correct segment
import numpy as np

def is_correct_timestamp(pred_start, pred_end, correct_segment):
    """
    Checks if the predicted clip [pred_start, pred_end] overlaps at least
    50% with the ground-truth clip [gt_start, gt_end] (i.e. `correct_segment`).

    Returns:
        1 if the overlap is >= 50% of the predicted clip's duration,
        0 otherwise.
    """

    gt_start, gt_end = correct_segment

    # Calculate intersection boundaries
    intersection_start = max(pred_start, gt_start)
    intersection_end   = min(pred_end, gt_end)

    # Lengths
    pred_length = max(0, pred_end - pred_start)
    intersection_length = max(0, intersection_end - intersection_start)

    # Edge case: if predicted clip or intersection is zero/negative, return 0
    if pred_length == 0 or intersection_length <= 0:
        return 0

    # Fraction of predicted clip that overlaps the ground-truth
    overlap_fraction = intersection_length / pred_length

    # Check if at least 50% of the predicted clip overlaps
    return int(overlap_fraction >= 0.9)



# Function to calculate Recall@k
def calculate_recall_at_k(results, k):
    correct_predictions = 0

    for result in results:
        # Parse the 'segment' (the ground-truth [start, end]) 
        # and 'top_k_timestamps' (the predicted clips).
        gt_segment = eval(result['segment'])  
        # e.g. [47, 60]

        top_k_list = eval(result['top_k_timestamps']) 
        # e.g. [ [47.2, 59.9], [67,89], [91,98] ] 
        # or maybe just one predicted clip [start, end].

        # Check if ANY of the top-k predicted clips is correct
        # (meaning at least 50% intersection, or your chosen criterion).
        # If yes, we consider this entire result "correct."
        is_correct_for_this_result = any(
            is_correct_timestamp(eval(pred_clip)[0], eval(pred_clip)[1], gt_segment)
            for pred_clip in top_k_list[:k]
        )

        if is_correct_for_this_result:
            correct_predictions += 1

    return correct_predictions / len(results)



# Load results and calculate Recall@k
results = pd.read_csv('results.csv')
temp = pd.read_csv('validation_youcook.csv')
filtered_results = results[results['video_id'].isin(temp['video_id'].unique())]

recall_at_1 = calculate_recall_at_k(filtered_results.to_dict('records'), 1)
recall_at_5 = calculate_recall_at_k(filtered_results.to_dict('records'), 5)
recall_at_10 = calculate_recall_at_k(filtered_results.to_dict('records'), 10)

print(f"Recall@1: {recall_at_1:.3f}")
print(f"Recall@5: {recall_at_5:.3f}")
print(f"Recall@10: {recall_at_10:.3f}")

# Recall@1: 0.319
# Recall@5: 0.654
# Recall@10: 0.754

# New baseline
# Recall@1: 0.318
# Recall@5: 0.644
# Recall@10: 0.749

Recall@1: 0.225
Recall@5: 0.357
Recall@10: 0.361


In [ ]:
import matplotlib.pyplot as plt

# Data for the top 5 models plus "Ours" (recall@1, 5, 10)
models = {
    'VAST (Finetuned)': [50.4, 74.3, 80.8],
    'UniVL + MELTR (Finetuned)': [33.7, 63.1, 74.8],
    'VideoCLIP (Finetuned)': [32.2, 62.6, 75.0],
    'MDMMT-2 (Finetuned)': [32.0, 64.0, 74.8],
    'TACo (Finetuned)': [29.6, 59.7, 72.7],
    'Ours (Zero-shot)':
    [recall_at_1 * 100, recall_at_5 * 100,
     recall_at_10 * 100]  # Example values, replace with your actual data
}

k_values = [1, 5, 10]

plt.figure(figsize=(12, 7))

for model, recalls in models.items():
    if model == 'Ours (Zero-shot)':
        plt.plot(k_values,
                 recalls,
                 marker='*',
                 markersize=12,
                 linewidth=3,
                 linestyle='--',
                 color='darkred',
                 label=model)
    else:
        plt.plot(k_values, recalls, marker='o', label=model)

plt.xlabel('K', fontsize=12)
plt.ylabel('Recall@K (%)', fontsize=12)
plt.title('YouCook2 Benchmark - Video Retrieval Performance', fontsize=14)
plt.legend(fontsize=10)
# plt.xscale('log')
plt.xticks(k_values, [str(k) for k in k_values])
plt.grid(True, which="both", ls="-", alpha=0.2)

plt.ylim(20, 85)  # Adjust y-axis limits for better visibility
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Data for the top 5 models plus "Ours" (recall@1, 5, 10)
models = {
    'Norton': [24.2, 51.9, 64.1],
    'VideoCLIP': [22.7, 50.4, 63.1],
    'VideoCOca': [20.3, 43.0, 53.3],
    'TACo': [19.9, 43.2, 55.7],
    'MIL-NCE': [15.1, 38.0, 51.2],
    'Ours': [recall_at_1 * 100, recall_at_5 * 100, recall_at_10 * 100]
}

k_values = [1, 5, 10]

plt.figure(figsize=(12, 7))

for model, recalls in models.items():
    if model == 'Ours':
        plt.plot(k_values,
                 recalls,
                 marker='*',
                 markersize=12,
                 linewidth=3,
                 linestyle='--',
                 color='darkred',
                 label=model)
    else:
        plt.plot(k_values, recalls, marker='o', label=model)

plt.xlabel('K', fontsize=12)
plt.ylabel('Recall@K (%)', fontsize=12)
plt.title('YouCook2 Benchmark - Zero-Shot Video Retrieval Performance',
          fontsize=14)
plt.legend(fontsize=10)
# plt.xscale('log')
plt.xticks(k_values, [str(k) for k in k_values])
plt.grid(True, which="both", ls="-", alpha=0.2)

plt.ylim(10, 80)  # Adjust y-axis limits for better visibility
plt.tight_layout()
plt.show()